In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as func
from PIL import Image
from datasetCreator import ImageSubset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from models import VGG

c:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
test = pd.read_csv('data/BNPP_DT_test_with_ages.csv')
val = pd.read_csv('data/BNPP_DT_val_with_ages.csv')
train = pd.read_csv('data/BNPP_DT_train_with_ages.csv')
f = h5py.File('data/bnpp_frontalonly_1024_10.hdf5')

3.669409867287782

In [5]:
#list(f.keys())
data = {}
ind=0
for key in list(f.keys()):
    image = f[key][:]
    #resizing to 224 x 224
    formatted = (image * 255 / np.max(image)).astype('uint8')
    img = Image.fromarray(formatted).resize((224,224))
    data[ind]=[key,img]
    #img_tensor = transform(img)
    #data[key] = img_tensor
    img.save(f'data/images/{key}.jpg')
    ind+=1

In [6]:
transform = transforms.Compose([
    transforms.PILToTensor()
])
train_data = ImageSubset(csv_file = 'data/BNPP_DT_train_with_ages.csv', data=data, img_dir='data/images/')
train_data

In [7]:
train_data[0]

{'image': tensor([[[ 13,  10,   7,  ...,   6,   6,   6],
          [ 13,  10,   7,  ...,   6,   6,   6],
          [ 13,  10,   7,  ...,   6,   6,   6],
          ...,
          [ 64,  53,  41,  ...,  24,  25,  25],
          [107,  93,  77,  ...,  59,  60,  61],
          [178, 163, 146,  ..., 133, 134, 135]]], dtype=torch.uint8),
 'Patient': 'Umefet_51429244_img1',
 'bnpp_log': 3.669409867287782}

In [11]:
train_dataloader = DataLoader(train_data,batch_size=4,shuffle=True,num_workers=4)

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VGG('VGG16').to(device)
model

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [16]:
x = torch.randn(2, 1, 224, 224)

In [18]:
out = model(x)
out

tensor([[-1.1104],
        [-0.9099]], grad_fn=<AddmmBackward0>)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[INFO]: Computation device: {device}")
epochs = 10
batch_size = 32

In [ ]:
# our transforms will differ a bit from the VGG paper
# as we are using the MNIST dataset, so, we will directly resize...
# ... the images to 224x224 and not crop them and we will not use...
# ... any random flippings also
train_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5), std=(0.5))])
valid_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5), std=(0.5))])